# Iterative Phase Estimation via the Cloud

This sample code and notebook was written by members of KPMG Quantum team in Australia and falls under an MIT License. It aims to demonstrate expanded capabilities of Basic Measurement Feedback targets and makes use of bounded loops, classical function calls at run time, nested conditional if statements, mid circuit measurements and qubit reuse. 


### Two Dimensional Inner Product Using Three Qubits

This notebook demonstrates an iterative phase estimation within Q#. The basic calculation it makes will be to calculate an inner product between two 2-dimensional vectors encoded on a target qubit and an ancilla qubit. An additional control qubit is also initialised, with a subsequent H gate applied. This control qubit will be used to readout the inner product via an iterative phase estimation. 

The circuit begins by encoding the pair of vectors on the target qubit and the ancilla qubit. It then applies an Oracle operator to the entire register, controlled off the control qubit. The Oracle operator generates a eigenphase on the target and ancilla qubit register, which when controlled generates a phase on the |1> state of the control qubit. This can then be read by applying another H gate to the controlled qubit to make the phase observable when measuring the Z projection.



In [ ]:
%azure.connect ""

##### Open Name Spaces Required
There are a number of Name Spaces required for this set of code. They are opened in the block below.

In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;

## Encoding vectors

The vectors v and c are to be encoded onto the target qubit. In two dimensions, the vectors are the angle about the Y axis on the Bloch Sphere. The state to be created is,

$$\ket{\Psi}=\frac{1}{\sqrt{2}}(\ket{+}\ket{v}+\ket{-}\ket{c},$$

which also takes the more readable form,

$$\ket{\Psi} = \frac{1}{2}(\ket{v}+\ket{c})\ket{0}+\frac{1}{2}(\ket{v}-\ket{c})\ket{1}.$$

Note that the angles represented as $\theta_1$ and $\theta_2$ are applied as $\frac{\theta_1}{2}$ and $\frac{\theta_2}{2}$. While the reason for this is not important, it is important to mention that coded values of  $\theta_1=0.0$ and $\theta_2=2.0\pi$ will calculate the inner product between vectors with actual angles $0$ and $\pi$ respectively (ie, anti-parallel).

In [ ]:
operation StateInitialisation(TargetReg : Qubit, AncilReg : Qubit, Theta1 : Double, Theta2 : Double) : Unit is Adj + Ctl { //This is state preperation operator A for encoding the 2D vector (page 7)
    H(AncilReg);

    Controlled R([AncilReg], (PauliY, -Theta1, TargetReg));        // Arbitrary controlled rotation based on theta. This is vector v.
                                                        
    X(AncilReg);                                                   // X gate on ancilla to change from |+> to |->.
    Controlled R([AncilReg], (PauliY, -Theta2, TargetReg));        // Arbitrary controlled rotation based on theta. This is vector c.
    X(AncilReg);                                                  
    H(AncilReg);                                                  
}

## The Oracle

An oracle G needs to be constructed such that it generates an eigenphase on the state encoded on the target qubit and the ancilla qubit. The construction of this oracle is unimportant to the demonstration within this notebook, but the operation it applies is,

$$G\ket \Psi = e^{2\pi i\theta} \ket \Psi.$$

where the inner product $\braket {v|c}$ is contained within $\theta$. When applied controlled on the control qubit which begins in that state $\ket{\Psi_\text{Control Qubit}} = \ket +$,

$$\begin{aligned}
    \text{Controlled }G \ket{\Psi_\text{Control Qubit}} \ket \Psi  & = \frac {1}{\sqrt{2}} (\ket 0 \ket \Psi + e^{2\pi i\theta}\ket 1 \ket \Psi )\\
    & =\frac {1}{\sqrt{2}} (\ket 0 + e^{2\pi i\theta}\ket 1) \ket \Psi
\end{aligned}$$

Now the control qubit contains the phase which relates to the inner product $\braket {v|c}$

$$\ket{\Psi_\text{Control Qubit}} = \frac {1}{\sqrt{2}} (\ket 0 + e^{2\pi i\theta}\ket 1)$$

In [ ]:
operation GOracle(TargetReg : Qubit, AncilReg : Qubit, Theta1 : Double, Theta2 : Double) : Unit is Adj + Ctl {
    Z(AncilReg);                                                      
    Adjoint StateInitialisation(TargetReg, AncilReg, Theta1, Theta2);
    X(AncilReg);                                                        // Apply X gates individually here as currently ApplyAll is not Adj + Ctl
    X(TargetReg);
    Controlled Z([AncilReg],TargetReg);
    X(AncilReg);
    X(TargetReg);
    StateInitialisation(TargetReg, AncilReg, Theta1, Theta2);         
}

## Iteration

Now for the iterative part of the circuit. For n measurements, consider that the phase can be represented as a binary value $\theta$, and that applying $2^n$ oracles makes the nth binary point of the phase observable (through simple binary multiplication, and modulus $2\pi$). The value of the control qubit can be readout, placed in a classical register and the qubit reset for use in the next iteration. The next iteration applies $2^{n-1}$ oracles, correcting phase on the control qubit dependent on the nth measurement. The state on the control qubit can be represented as,

$$ \ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i\theta}\ket 1 $$

where $\theta = 0.\theta_0\theta_1\theta_2\theta_3$...

Applying $2^n$ controlled oracles gives the state on the control qubit,

$$ G^{2^n}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\theta_n\theta_{n+1}\theta_{n+2}\theta_{n+3}...}\ket 1 $$

Consider that the phase has no terms deeper than $\theta_n$ (ie, terms $\theta_{n+1},\theta_{n+2}, \text{etc}$),

$$ G^{2^n}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\theta_n}\ket 1 $$

Now the value $\theta_n$ can be observed with a H gate and a measurement projecting along the Z axis. Resetting the control qubit and applying the oracle $2^{n-1}$ times,

$$ G^{2^{n-1}}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\theta_{n-1}\theta_n}\ket 1 $$

Using the previous measured value for $\theta_n$, the additional binary point can be rotated out.

$$ RZ(-2\pi \times 0.0\theta_n)G^{n-1}\ket {\Psi_{\text{Control Qubit}}} = \ket 0 + e^{2\pi i 0.\theta_{n-1}}\ket 1 $$

This process is iteratively applied for some bit precision n to obtain the phase $0.\theta_0\theta_1\theta_2...\theta_{n}$. The value is stored as a binary value $x = \theta_0\theta_1\theta_2...\theta_{n}$ as only integers are manipulatable at runtime currently.

As the readout tells nothing of either vector, only the inner product between them, the states on the target qubit and ancilla qubit <u>remain in the same state</u> throughout the process!

In [ ]:
operation IterativePhaseEstimation(TargetReg : Qubit, AncilReg : Qubit, Theta1 : Double, Theta2 : Double, Measurements : Int) : Int{
    use ControlReg = Qubit();
    mutable MeasureControlReg = ConstantArray(Measurements, Zero);                                  //Set up array of measurement results of zero
    mutable bitValue = 0;
    StateInitialisation(TargetReg, AncilReg, Theta1, Theta2);                                       //Apply to initialise state, this is defined by the angles theta1 and theta2
    for index in 0 .. Measurements - 1{                                                             
        H(ControlReg);                                                                              
        if index > 0 {                                                                              //Don't apply rotation on first set of oracles
            for index2 in 0 .. index - 1{                                                           //Loop through previous results
                if MeasureControlReg[Measurements - 1 - index2] == One{                             
                    R(PauliZ, -IntAsDouble(2^(index2))*PI()/(2.0^IntAsDouble(index)), ControlReg);  
                }
            }
            
        }
        let powerIndex = (1 <<< (Measurements - 1 - index));
        for _ in 1 .. powerIndex{                                                                   //Apply a number of oracles equal to 2^index, where index is the number or measurements left
                Controlled GOracle([ControlReg],(TargetReg, AncilReg, Theta1, Theta2));
            }
        H(ControlReg);
        set MeasureControlReg w/= (Measurements - 1 - index) <- MResetZ(ControlReg);                //Make a measurement mid circuit 
        if MeasureControlReg[Measurements - 1 - index] == One{
            set bitValue += 2^(index);                                                              //Assign bitValue based on previous measurement
                                                                                                    
        }
    }
    Reset(ControlReg);                                                                              //Reset qubits for end of circuit.
    Reset(TargetReg);                                           
    Reset(AncilReg); 
    return bitValue;
}

Finally to calculate the inner product from the measured value,

$$\braket {v|c} = -cos(2\pi x / 2^n)$$

where $x = \theta_0\theta_1\theta_2...\theta_{n}$. The denominator within the cosine function is to shift the binary point to match the original value $\theta$.

**Note**: For inner product that are not -1 or 1, the solutions are paired with a value difference of $2^{n-1}$. For example for n=3 measurements, the measured bit value of 2 would also have a pair solution of 6. Either of these values produce the same value of the inner product when input as the variable to the even function cosine (resulting in an inner product of 0 in this example).

**Note**: For inner product solutions between the discrete bit precision, a distribution of results will be produced based on where the inner product lies between the discrete bit value. 

In [ ]:
operation SimulateInnerProduct() : Int{
    let Theta1 = 0.0;                                                                           //Specify the angles for inner product
    let Theta2 = 0.0;
    let Measurements = 3;                                                                       //Specify the bit resolution in the iterative phase estimation
                                                                                                //For Jobs on hardware the suggested number of measurements is 3
    use TargetReg = Qubit();                                                                    //TargetReg has states v and c qubits contained on it
    use AncilReg = Qubit();                                                                     //Create ancilla
    let Results = IterativePhaseEstimation(TargetReg, AncilReg, Theta1, Theta2, Measurements);  //This runs iterative phase estimation
    

    let DoubleVal = PI() * IntAsDouble(Results) / IntAsDouble(2 ^ (Measurements-1));
    let InnerProductValue = -Cos(DoubleVal);                                                      //Convert to the final inner product
    Message("The Inner Product is:");
    Message($"{InnerProductValue}");
    Message("The True Inner Product is:");
    Message($"{Cos(Theta1/2.0)*Cos(Theta2/2.0)+Sin(Theta1/2.0)*Sin(Theta2/2.0)}");
    Message("The Bit Value measured is");

                                                                            
    return Results;                                                                             //Return Measured values
}

In [ ]:
%simulate SimulateInnerProduct

The inner product operation is reconstructed to fit within the requirements of the target. In this case, calls like "Message" and manipulation of doubles is not allowed. Therefore the output will be the bit value as generated by the IterativePhaseEstimation operation.

In [ ]:
operation InnerProduct() : Int{
    let Theta1 = 0.0;                                                                           //Specify the angles for inner product
    let Theta2 = 0.0;
    let Measurements = 3;                                                                       //Specify the bit resolution in the iterative phase estimation
                                                                                                //For Jobs on hardware the suggested number of measurements is 3
    use TargetReg = Qubit();                                                                    //TargetReg has states v and c qubits contained on it
    use AncilReg = Qubit();                                                                     //Create ancilla
    let Results = IterativePhaseEstimation(TargetReg, AncilReg, Theta1, Theta2, Measurements);  //This runs iterative phase estimation
                                                                           
    return Results;                                                                             //Return Measured values
}

Specify the target.

**Note**: The target requires a target execution profile that supports basic measurement feedback.

In [ ]:
%azure.target quantinuum.sim.h1-1e

In [ ]:
%azure.target-capability AdaptiveExecution

Submit the job to the target. The circuit is approximately 1.2 HQC each shot. The number of shots specified is 128, but this can be increased to reduce the variance of the result, up to some stable distribution.

**Note**: Choosing input parameters which only has one solution state (inner produces of -1 or 1) are ideal for visibility at a low number of shots.

In [ ]:
%azure.submit InnerProduct shots = 128


Check the status of the job specified.

In [ ]:
%azure.status

When the job is complete, output a histogram of results.

The results should show a majority in the solution states.

In [ ]:
%azure.output